# Install

In [ ]:
!git clone https://github.com/farzane-maghsoudi/TTL-GAN
import os
os.chdir('TTL-GAN/')
!pip install -r requirements.txt
!bash ./download_dataset.sh horse2zebra

#dataset

In [ ]:
!bash ./download_dataset.sh horse2zebra

#Training

In [ ]:
!python main.py --dataset horse2zebra --light True

Restoring from the previous checkpoint

In [ ]:
!python main.py --dataset horse2zebra --resume True --light True

#Testing

In [ ]:
!python main.py --dataset mini --phase test --light True

#Copy checkpoint to GoogleDrive and convert




In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!mkdir ./results

In [ ]:
b = "/content/gdrive/MyDrive/checkpointTTL-GAN/horse2zebra_params_latest.pt"
c = "/content/TTL-GAN/results/horse2zebra_params_latest.pt"
f1 = "/content/gdrive/MyDrive/checkpointTTL-GAN/horse2zebra_params_0002000.pt"
f2 = "/content/TTL-GAN/results/horse2zebra/model/horse2zebra_params_0002000.pt"
cc = "/content/TTL-GAN/results/horse2zebra_params_0002000.pt"

Colab to Drive

In [ ]:
import shutil
shutil.copy( c , b )
shutil.copy( f2 , f1 )

Drive to Colab

In [ ]:
import shutil
shutil.copy( f1 , cc )

'/content/TTL-GAN/results/horse2zebra_params_0002000.pt'

#Test save

In [ ]:
#import shutil
#b = "/content/gdrive/MyDrive/results/501/UGATIT_light_apple2orange_lsgan_4resblock_6dis_1_1_10_10_1000_1_1_1_sn_smoothing"
#c = "/content/UG/results/UGATIT_light_apple2orange_lsgan_4resblock_6dis_1_1_10_10_1000_1_1_1_sn_smoothing"
!cp "/content/UG/results/UGATIT_light_apple2orange_lsgan_4resblock_6dis_1_1_10_10_1000_1_1_1_sn_smoothing" -r "/content/gdrive/MyDrive/results"

In [ ]:
n_blocks = 8
for i in range(2, n_blocks+1):
  if i%3 == 0:
    x2 = 3
    print (i, x2)
  if i%3 == 1:
    x3 = 4
    print (i, x3)
  if i%3 == 2:
    if i> 4:
      x4 = 10
      print (i, x4)
    else:
      x4 = 5
      print (i, x4)

In [ ]:
import torch
import torch.nn as nn
a = torch.randn(1,256, 64, 64)
#a = torch.reshape(a, (256, 64, 64))
atten  = torch.nn.MultiheadAttention(64, 8)
relu = torch.nn.ReLU(True)
class MultiSelfAttentionBlock(nn.Module):
    def __init__(self, atten, relu, dim =64, featur = 256):
      super(MultiSelfAttentionBlock, self).__init__()
      self.dim = dim
      self.featur = featur
      self.atten  = atten
      self.relu  = relu


    def forward(self, x):
      print("as")
      out = torch.reshape(x, (self.featur, self.dim, self.dim))
      out, _ = self.atten(out, out, out)
      out = self.relu(torch.reshape(out, (1, self.featur, self.dim, self.dim)))
      print("a")

      return out

#out, _ = atten(a, a, a)
#out = torch.reshape(out, (1, 256, 64, 64))
for i in range(3):
  out = MultiSelfAttentionBlock(a, atten, relu)
  #print (out.shape)

In [ ]:
import torch
import torch.nn as nn
a = torch.randn(1,256, 64, 64)
atten  = torch.nn.MultiheadAttention(64, 8)
relu = torch.nn.ReLU(True)
for i in range(3):
  print("as")
  out = torch.reshape(a, (256, 64, 64))
  out, _ = atten(out, out, out)
  out = torch.reshape(out, (1, 256, 64, 64))
  out = relu(out)
  print("a")
  print (out.shape)

In [ ]:
import torch
import torch.nn as nn
x = torch.randn(1,256, 64, 64)
atten  = torch.nn.MultiheadAttention(64, 8)
relu = torch.nn.ReLU(True)
n_blocks = 8
out = torch.reshape(x, (256, 64, 64))
out, _ = atten(out, out, out)
out = relu(torch.reshape(out, (1, 256, 64, 64)))
xa1 = out
xa11 = out
if n_blocks>1:
  for i in range(2, 8+1):
    if i%3 == 2:
      out = torch.reshape(out, (256, 64, 64))
      out, _ = atten(out, out, out)
      xa2 = out = relu(torch.reshape(out, (1, 256, 64, 64)))
      print (i)
    elif i%3 == 0:
      out = torch.reshape(out + xa1, (256, 64, 64))
      out, _ = atten(out, out, out)
      xa3 = out = relu(torch.reshape(out, (1, 256, 64, 64)))
      print (i)
    elif i < 6:
      out = torch.reshape(out + xa1 + xa2, (256, 64, 64))
      out, _ = atten(out, out, out)
      xa1 = out = relu(torch.reshape(out, (1, 256, 64, 64)))
      print (i)
    else:
      out = torch.reshape(out + xa1 + xa2 + xa11, (256, 64, 64))
      out, _ = atten(out, out, out)
      xa11 = xa1
      xa1 = out = relu(torch.reshape(out, (1, 256, 64, 64)))
      print (i)

In [ ]:
import torch
import torch.nn as nn
a = torch.randn(1,128, 56, 56)
b = torch.randn(1,128, 28, 28)
c = torch.randn(1,128, 14, 14)
up1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
up2 = nn.Upsample(scale_factor=4, mode='bilinear', align_corners=True)
pad1 = nn.ReflectionPad2d(4)
pad2 = nn.ReflectionPad2d(4)
pad3 = nn.ReflectionPad2d(4)
a = pad1 (a)
b = up1(b)
b = pad2(b)
c = up2(c)
c = pad3(c)
print (a.shape)
print(b.shape)
print(c.shape)

In [ ]:
self.save(self.checkpoint_dir, counter)
				    a = (epoch+1)*1000
				    b = "/content/gdrive/MyDrive/checkpoint/UGATIT_light_apple2orange_lsgan_4resblock_6dis_1_1_10_10_1000_1_1_1_sn_smoothing"
				    c = "/content/UG/checkpoint/UGATIT_light_apple2orange_lsgan_4resblock_6dis_1_1_10_10_1000_1_1_1_sn_smoothing"
				    f1 = "/UGATIT_light.model-"+ str(a)+ ".index"
				    f2 = "/UGATIT_light.model-"+ str(a)+ ".meta"
				    f3 = "/UGATIT_light.model-" + str(a)+ ".data-00000-of-00001"
				    f = "/checkpoint"
				    import shutil
				    shutil.copy( c+f1 , b+f1 )
				    shutil.copy( c+f2 , b+f2 )
				    shutil.copy( c+f3 , b+f3 )
				    shutil.copy( c+f , b+f )

In [ ]:
import torch
import torch.nn as nn
input = torch.randn(1,256, 256,3)
S = torch.linalg.svdvals(input)
#out = torch.reshape(s1[0,:,0], (-1,))
R = torch.reshape(S[0,:,0], (-1,))
G = torch.reshape(S[0,:,1], (-1,))
B = torch.reshape(S[0,:,2], (-1,))

c1 = torch.reshape(S[0,:,0], (-1,))*torch.reshape(S[0,:,1], (-1,))
c2 = torch.cartesian_prod(torch.reshape(S[0,:,0], (-1,)), torch.reshape(S[0,:,2], (-1,)))
c3 = torch.cartesian_prod(torch.reshape(S[0,:,1], (-1,)), torch.reshape(S[0,:,2], (-1,)))
c4 = torch.cartesian_prod(torch.reshape(S[0,:,0], (-1,)), torch.reshape(S[0,:,1], (-1,)), torch.reshape(S[0,:,2], (-1,)))
R1 = torch.cartesian_prod(R, G)
print(c1.shape)
print(c2.shape)
print(c3.shape)
print(c4.shape)
print(R.shape)
print(G.shape)
print(B.shape)
print(R1.shape)

In [ ]:
import torch.nn as nn
input = torch.randn(1,256, 256,3)
S = torch.linalg.svdvals(input)
K = 32

c1 = torch.matmul(torch.reshape(S[0,0:K,0], (K,1)), torch.reshape(S[0,0:K,1], (1,K)))
c2 = torch.matmul(torch.reshape(S[0,0:K,1], (K,1)), torch.reshape(S[0,0:K,2], (1,K)))
c3 = torch.matmul(torch.reshape(S[0,0:K,0], (K,1)), torch.reshape(S[0,0:K,2], (1,K)))
c4 = torch.matmul(torch.reshape(c1, (K, K, 1)), torch.reshape(S[0,0:K,2], (1, 1, K)))

svd = torch.cat((torch.reshape(c1, (1,1,K,K)),torch.reshape(c2, (1,1,K,K)),torch.reshape(c3, (1,1,K,K))), dim=1)
print(svd.shape)
svd = torch.cat((svd,torch.reshape(c4, (1,K,K,K))), dim=1)
print(svd.shape)

In [ ]:
pip install involution_pytorch

In [ ]:
import torch
from involution_pytorch import Inv2d
import torch.nn as nn
inv = Inv2d(
    channels=16,
    kernel_size=3,
    stride=1
)

x = torch.rand(1, 16, 32, 32)
#y = inv(x) # [1, 16, 32, 32]
#print (y.shape)
m = nn.Conv2d(16, 33, 3, stride=2)
output = m(x)
print (output.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial
from timm.models.layers import DropPath, trunc_normal_
import math

class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.dwconv = DWConv(hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def forward(self, x, H, W):
        x = self.fc1(x)
        x = self.act(x + self.dwconv(x, H, W))
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

def local_conv(dim):
    return nn.Conv2d(dim, dim, kernel_size=3, padding=1, stride=1, groups=dim)

class Attention(nn.Module):
    def __init__(self, dim, mask, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., sr_ratio=1, linear=False):
        super().__init__()
        assert dim % num_heads == 0, f"dim {dim} should be divided by num_heads {num_heads}."

        self.dim = dim
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5
        self.sr_ratio=sr_ratio
        if sr_ratio>1:
            if mask:
                self.q = nn.Linear(dim, dim, bias=qkv_bias)
                self.kv1 = nn.Linear(dim, dim, bias=qkv_bias)
                self.kv2 = nn.Linear(dim, dim, bias=qkv_bias)
                if self.sr_ratio==8:
                    f1, f2, f3 = 14*14, 56, 28
                elif self.sr_ratio==4:
                    f1, f2, f3 = 49, 14, 7
                elif self.sr_ratio==2:
                    f1, f2, f3 = 2, 1, None
                self.f1 = nn.Linear(f1, 1)
                self.f2 = nn.Linear(f2, 1)
                if f3 is not None:
                    self.f3 = nn.Linear(f3, 1)
            else:
                self.sr = nn.Conv2d(dim, dim, kernel_size=sr_ratio, stride=sr_ratio)
                self.norm = nn.LayerNorm(dim)
                self.act = nn.GELU()

                self.q1 = nn.Linear(dim, dim//2, bias=qkv_bias)
                self.kv1 = nn.Linear(dim, dim, bias=qkv_bias)
                self.q2 = nn.Linear(dim, dim // 2, bias=qkv_bias)
                self.kv2 = nn.Linear(dim, dim, bias=qkv_bias)
        else:
            self.q = nn.Linear(dim, dim, bias=qkv_bias)
            self.kv = nn.Linear(dim, dim * 2, bias=qkv_bias)

        self.lepe_linear = nn.Linear(dim, dim)
        self.lepe_conv = local_conv(dim)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        self.linear = linear
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def forward(self, x, H, W, mask):
        B, N, C = x.shape
        lepe = self.lepe_conv(
            self.lepe_linear(x).transpose(1, 2).view(B, C, H, W)).view(B, C, -1).transpose(-1, -2)
        if self.sr_ratio > 1:
            if mask is None:
                # global
                q1 = self.q1(x).reshape(B, N, self.num_heads//2, C // self.num_heads).permute(0, 2, 1, 3)
                x_ = x.permute(0, 2, 1).reshape(B, C, H, W)
                x_1 = self.sr(x_).reshape(B, C, -1).permute(0, 2, 1)
                x_1 = self.act(self.norm(x_1))
                kv1 = self.kv1(x_1).reshape(B, -1, 2, self.num_heads//2, C // self.num_heads).permute(2, 0, 3, 1, 4)
                k1, v1 = kv1[0], kv1[1] #B head N C

                attn1 = (q1 @ k1.transpose(-2, -1)) * self.scale #B head Nq Nkv
                attn1 = attn1.softmax(dim=-1)
                attn1 = self.attn_drop(attn1)
                x1 = (attn1 @ v1).transpose(1, 2).reshape(B, N, C//2)

                global_mask_value = torch.mean(attn1.detach().mean(1), dim=1) # B Nk  #max ?  mean ?
                global_mask_value = F.interpolate(global_mask_value.view(B,1,H//self.sr_ratio,W//self.sr_ratio),
                                                  (H, W), mode='nearest')[:, 0]

                # local
                q2 = self.q2(x).reshape(B, N, self.num_heads // 2, C // self.num_heads).permute(0, 2, 1, 3) #B head N C
                kv2 = self.kv2(x_.reshape(B, C, -1).permute(0, 2, 1)).reshape(B, -1, 2, self.num_heads // 2,
                                                                          C // self.num_heads).permute(2, 0, 3, 1, 4)
                k2, v2 = kv2[0], kv2[1]
                q_window = 7
                window_size= 7
                q2, k2, v2 = window_partition(q2, q_window, H, W), window_partition(k2, window_size, H, W), \
                             window_partition(v2, window_size, H, W)
                attn2 = (q2 @ k2.transpose(-2, -1)) * self.scale
                # (B*numheads*num_windows, window_size*window_size, window_size*window_size)
                attn2 = attn2.softmax(dim=-1)
                attn2 = self.attn_drop(attn2)

                x2 = (attn2 @ v2)  # B*numheads*num_windows, window_size*window_size, C   .transpose(1, 2).reshape(B, N, C)
                x2 = window_reverse(x2, q_window, H, W, self.num_heads // 2)

                local_mask_value = torch.mean(attn2.detach().view(B, self.num_heads//2, H//window_size*W//window_size, window_size*window_size, window_size*window_size).mean(1), dim=2)
                local_mask_value = local_mask_value.view(B, H // window_size, W // window_size, window_size, window_size)
                local_mask_value=local_mask_value.permute(0, 1, 3, 2, 4).contiguous().view(B, H, W)

                # mask B H W
                x = torch.cat([x1, x2], dim=-1)
                x = self.proj(x+lepe)
                x = self.proj_drop(x)
                # cal mask
                mask = local_mask_value+global_mask_value
                mask_1 = mask.view(B, H * W)
                mask_2 = mask.permute(0, 2, 1).reshape(B, H * W)
                mask = [mask_1, mask_2]
            else:
                q = self.q(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)

                # mask [local_mask global_mask]  local_mask [value index]  value [B, H, W]
                # use mask to fuse
                mask_1, mask_2 = mask
                mask_sort1, mask_sort_index1 = torch.sort(mask_1, dim=1)
                mask_sort2, mask_sort_index2 = torch.sort(mask_2, dim=1)
                if self.sr_ratio == 8:
                    token1, token2, token3 = H * W // (14 * 14), H * W // 56, H * W // 28
                    token1, token2, token3 = token1 // 4, token2 // 2, token3 // 4
                elif self.sr_ratio == 4:
                    token1, token2, token3 = H * W // 49, H * W // 14, H * W // 7
                    token1, token2, token3 = token1 // 4, token2 // 2, token3 // 4
                elif self.sr_ratio == 2:
                    token1, token2 = H * W // 2, H * W // 1
                    token1, token2 = token1 // 2, token2 // 2
                if self.sr_ratio==4 or self.sr_ratio==8:
                    p1 = torch.gather(x, 1, mask_sort_index1[:, :H * W // 4].unsqueeze(-1).repeat(1, 1, C))  # B, N//4, C
                    p2 = torch.gather(x, 1, mask_sort_index1[:, H * W // 4:H * W // 4 * 3].unsqueeze(-1).repeat(1, 1, C))
                    p3 = torch.gather(x, 1, mask_sort_index1[:, H * W // 4 * 3:].unsqueeze(-1).repeat(1, 1, C))
                    seq1 = torch.cat([self.f1(p1.permute(0, 2, 1).reshape(B, C, token1, -1)).squeeze(-1),
                                      self.f2(p2.permute(0, 2, 1).reshape(B, C, token2, -1)).squeeze(-1),
                                      self.f3(p3.permute(0, 2, 1).reshape(B, C, token3, -1)).squeeze(-1)], dim=-1).permute(0,2,1)  # B N C

                    x_ = x.view(B, H, W, C).permute(0, 2, 1, 3).reshape(B, H * W, C)
                    p1_ = torch.gather(x_, 1, mask_sort_index2[:, :H * W // 4].unsqueeze(-1).repeat(1, 1, C))  # B, N//4, C
                    p2_ = torch.gather(x_, 1, mask_sort_index2[:, H * W // 4:H * W // 4 * 3].unsqueeze(-1).repeat(1, 1, C))
                    p3_ = torch.gather(x_, 1, mask_sort_index2[:, H * W // 4 * 3:].unsqueeze(-1).repeat(1, 1, C))
                    seq2 = torch.cat([self.f1(p1_.permute(0, 2, 1).reshape(B, C, token1, -1)).squeeze(-1),
                                      self.f2(p2_.permute(0, 2, 1).reshape(B, C, token2, -1)).squeeze(-1),
                                      self.f3(p3_.permute(0, 2, 1).reshape(B, C, token3, -1)).squeeze(-1)], dim=-1).permute(0,2,1)  # B N C
                elif self.sr_ratio==2:
                    p1 = torch.gather(x, 1, mask_sort_index1[:, :H * W // 2].unsqueeze(-1).repeat(1, 1, C))  # B, N//4, C
                    p2 = torch.gather(x, 1, mask_sort_index1[:, H * W // 2:].unsqueeze(-1).repeat(1, 1, C))
                    seq1 = torch.cat([self.f1(p1.permute(0, 2, 1).reshape(B, C, token1, -1)).squeeze(-1),
                                      self.f2(p2.permute(0, 2, 1).reshape(B, C, token2, -1)).squeeze(-1)], dim=-1).permute(0, 2, 1)  # B N C

                    x_ = x.view(B, H, W, C).permute(0, 2, 1, 3).reshape(B, H * W, C)
                    p1_ = torch.gather(x_, 1, mask_sort_index2[:, :H * W // 2].unsqueeze(-1).repeat(1, 1, C))  # B, N//4, C
                    p2_ = torch.gather(x_, 1, mask_sort_index2[:, H * W // 2:].unsqueeze(-1).repeat(1, 1, C))
                    seq2 = torch.cat([self.f1(p1_.permute(0, 2, 1).reshape(B, C, token1, -1)).squeeze(-1),
                                      self.f2(p2_.permute(0, 2, 1).reshape(B, C, token2, -1)).squeeze(-1)], dim=-1).permute(0, 2, 1)  # B N C

                kv1 = self.kv1(seq1).reshape(B, -1, 2, self.num_heads // 2, C // self.num_heads).permute(2, 0, 3, 1, 4) # kv B heads N C
                kv2 = self.kv2(seq2).reshape(B, -1, 2, self.num_heads // 2, C // self.num_heads).permute(2, 0, 3, 1, 4)
                kv = torch.cat([kv1, kv2], dim=2)
                k, v = kv[0], kv[1]
                attn = (q @ k.transpose(-2, -1)) * self.scale
                attn = attn.softmax(dim=-1)
                attn = self.attn_drop(attn)

                x = (attn @ v).transpose(1, 2).reshape(B, N, C)
                x = self.proj(x+lepe)
                x = self.proj_drop(x)
                mask=None

        else:
            q = self.q(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
            kv = self.kv(x).reshape(B, -1, 2, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)

            k, v = kv[0], kv[1]

            attn = (q @ k.transpose(-2, -1)) * self.scale
            attn = attn.softmax(dim=-1)
            attn = self.attn_drop(attn)
            x = (attn @ v).transpose(1, 2).reshape(B, N, C)
            x = self.proj(x+lepe)
            x = self.proj_drop(x)
            mask=None

        return x, mask

def window_partition(x, window_size, H, W):
    B, num_heads, N, C = x.shape
    x = x.contiguous().view(B*num_heads, N, C).contiguous().view(B*num_heads, H, W, C)
    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C).\
        view(-1, window_size*window_size, C)
    return windows  #(B*numheads*num_windows, window_size, window_size, C)


def window_reverse(windows, window_size, H, W, head):
    Bhead = int(windows.shape[0] / (H * W / window_size / window_size))
    x = windows.view(Bhead, H // window_size, W // window_size, window_size, window_size, -1)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(Bhead, H, W, -1).view(Bhead//head, head, H, W, -1)\
        .contiguous().permute(0,2,3,1,4).contiguous().view(Bhead//head, H, W, -1).view(Bhead//head, H*W, -1)
    return x #(B, H, W, C)

class Block(nn.Module):

    def __init__(self, dim, mask, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm, sr_ratio=1, linear=False):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(
            dim, mask,
            num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale,
            attn_drop=attn_drop, proj_drop=drop, sr_ratio=sr_ratio, linear=linear)
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def forward(self, x, H, W, mask):
        x_, mask = self.attn(self.norm1(x), H, W, mask)
        x = x + self.drop_path(x_)
        x = x + self.drop_path(self.mlp(self.norm2(x), H, W))

        return x, mask

class DWConv(nn.Module):
    def __init__(self, dim=768):
        super(DWConv, self).__init__()
        self.dwconv = nn.Conv2d(dim, dim, 3, 1, 1, bias=True, groups=dim)

    def forward(self, x, H, W):
        B, N, C = x.shape
        x = x.transpose(1, 2).view(B, C, H, W)
        x = self.dwconv(x)
        x = x.flatten(2).transpose(1, 2)

        return x

block1 = Block(dim=embed_dims[i], mask=True if (j%2==1 and i<num_stages-1) else False, num_heads=num_heads[i],
                mlp_ratio=mlp_ratios[i], qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[cur + j], norm_layer=norm_layer,
                sr_ratio=sr_ratios[i], linear=linear)

x = torch.rand(1, 16, 32, 32)
_, _, H, W = x.shape
mask= None
x, mask = block1(x, H, W, mask)

In [ ]:
dim=embed_dims[i] #
mask=True if (j%2==1 and i<num_stages-1) else False #
num_heads=num_heads[i] #
mlp_ratio=mlp_ratios[i] #
qkv_bias=qkv_bias #
qk_scale=qk_scale #
drop=drop_rate #
attn_drop=attn_drop_rate #
drop_path=dpr[cur + j] #
norm_layer=norm_layer #
sr_ratio=sr_ratios[i] #
linear=linear #
